In [ ]:
#Importando librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import constants

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/02. Riesgo Instituciones/Riesgo de crédito/Data_score.csv', sep=";",thousands='.', decimal=',')
df.sample(10)

**Variables**

ID: Identificación del cliente

DEST_CRED: Destino en el que se va a usar el crédito

SEXO: Género (Hombre o Mujer)

CATEGORIA_NEG: Tipo de Negocio

EXPERIENCIA_NEGOCIO: Meses de experiencia o antigüedad en el negocio

ESTADO_CIVIL: Estado civil del cliente

EDAD: Edad del cliente

NIVEL_ENDEUDAMIENTO: % de endeudamiento del cliente

NUMERO_ENTIDADES: Con cuántas entidades financieras el cliente tiene un crédito.

M_MAX: Día máximo en que el cliente se ha demorado en pagar una de sus cuotas, si no hay demora el valor por defecto es 5500.

ING_BRUTO: Ingreso Bruto estimado del cliente.

In [ ]:
df.describe().round(1)

**Análisis de datos cuantitativos**

In [ ]:
fig = plt.figure(figsize=(15,15))
ax = fig.gca()
df.hist(ax = ax)
plt.show()

**Análisis de datos cualitativos**

In [ ]:
df.select_dtypes(include = [object]).columns

In [ ]:
df.groupby("DEST_CRED").size().sort_values()

In [ ]:
sns.countplot(x="DEST_CRED", data=df)

In [ ]:
df.groupby("SEXO").size().sort_values()

In [ ]:
df.groupby("CATEGORIA_NEG").size().sort_values()

In [ ]:
df.groupby("ESTADO_CIVIL").size().sort_values()

In [ ]:
df.groupby("NUMERO_ENTIDADES").size().sort_values()

**Tratamiento de datos nulos**

In [ ]:
from statistics import mode

In [ ]:
df.isnull().sum()

In [ ]:
#Hallando moda de la variable "DEST_CRED"
moda_DEST_CRED = mode(df['DEST_CRED'])
moda_DEST_CRED

In [ ]:
df['DEST_CRED']=df['DEST_CRED'].fillna(moda_DEST_CRED)

In [ ]:
#Hallando moda de la variable "DEST_CRED"
moda_CATEGORIA_NEG = mode(df['CATEGORIA_NEG'])
moda_CATEGORIA_NEG

In [ ]:
df['CATEGORIA_NEG']=df['CATEGORIA_NEG'].fillna(moda_CATEGORIA_NEG)

In [ ]:
df.isnull().sum()

**Transformación de datos**

In [ ]:
df.select_dtypes(include=["object"]).columns

In [ ]:
df['DEST_CRED'].unique().tolist()

In [ ]:
def sust_d(DEST_CRED):
  if DEST_CRED == 'Capital de trabajo':
    return 0
  if DEST_CRED == 'Activo fijo':
    return 1
  if DEST_CRED == 'Consumo':
    return 2
  if DEST_CRED == 'Crédito vehículo':
    return 3
df['DEST_CRED'] = df['DEST_CRED'].apply(lambda x: sust_d(x))

In [ ]:
df['SEXO'].unique().tolist()

In [ ]:
def sust_g(SEXO):
  if SEXO == 'Mujer':
    return 0
  if SEXO == 'Hombre':
    return 1
df['SEXO'] = df['SEXO'].apply(lambda x: sust_g(x))

In [ ]:
df['CATEGORIA_NEG'].unique().tolist()

In [ ]:
def sust_c(CATEGORIA_NEG):
  if CATEGORIA_NEG == 'Comercio':
    return 0
  if CATEGORIA_NEG == 'Producción':
    return 1
  if CATEGORIA_NEG == 'Servicios':
    return 2
  if CATEGORIA_NEG == 'Emprendedor':
    return 3
  if CATEGORIA_NEG == 'Informal':
    return 4
df['CATEGORIA_NEG'] = df['CATEGORIA_NEG'].apply(lambda x: sust_c(x))

In [ ]:
df['ESTADO_CIVIL'].unique().tolist()

In [ ]:
def sust_e(ESTADO_CIVIL):
  if ESTADO_CIVIL == 'Soltero':
    return 0
  if ESTADO_CIVIL == 'Unión Libre':
    return 1
  if ESTADO_CIVIL == 'Casado':
    return 2
  if ESTADO_CIVIL == 'Divorciado':
    return 3
  if ESTADO_CIVIL == 'Viudo':
    return 4
df['ESTADO_CIVIL'] = df['ESTADO_CIVIL'].apply(lambda x: sust_e(x))

In [ ]:
df['NUMERO_ENTIDADES'].unique().tolist()

In [ ]:
def sust_n(NUMERO_ENTIDADES):
  if NUMERO_ENTIDADES == 'uno':
    return "1"
  if NUMERO_ENTIDADES == 'dos':
    return "2"
  if NUMERO_ENTIDADES == 'tres':
    return "3"
  if NUMERO_ENTIDADES == 'cuatro':
    return "4"
  if NUMERO_ENTIDADES == 'cinco o +':
    return "5"
df['NUMERO_ENTIDADES'] = df['NUMERO_ENTIDADES'].apply(lambda x: sust_n(x))

In [ ]:
df.drop(["ID"],axis =1, inplace = True)

In [ ]:
df.sample(10)

In [ ]:
df.info()

In [ ]:
df['NUMERO_ENTIDADES']= df['NUMERO_ENTIDADES'].astype('int64')

In [ ]:
df.info()

**Definición del Target**

In [ ]:
df_target=df.copy()
df_target

In [ ]:
dias = int(input("Máximo de días que la Entidad permite que se atrasen sus clientes "))
dias

In [ ]:
#Definiendo el umbral de default en días
# Default 1
# No Default 0
df_target['Default'] = np.where(df_target['M_MAX']>dias,'1','0')

In [ ]:
df_target.sample(5)

In [ ]:
df_target.groupby('Default').size().sort_values()

In [ ]:
sns.countplot(x='Default', data=df_target)

In [ ]:
df_target['Default'] = df_target['Default'].astype(int)

**Análisis Univariante**

**Escalado de datos**

In [ ]:
df_escalado = df_target.copy()
df_escalado

In [ ]:
  #Visualización sin escalar
  sns.kdeplot(data=df_escalado)
  plt.show()

In [ ]:
#Dividiendo características y target
df_escalado.shape

In [ ]:
array = df_escalado.values
X_clas = array[:,0:10]
Y_clas = array[:,10]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
#Seleccionando features normalizadas prescindiendo de "Default"
features=X_clas
scaler=MinMaxScaler(feature_range=(0,1))
rescaledx =scaler.fit_transform(features)
rescaledx

In [ ]:
columns=list(df_escalado.columns)
columns

In [ ]:
df_scaled = pd.DataFrame(data=rescaledx, columns =['DEST_CRED','SEXO','CATEGORIA_NEG','EXPERIENCIA_NEGOCIO',
                                                 'ESTADO_CIVIL','EDAD','NIVEL_ENDEUDAMIENTO', 'NUMERO_ENTIDADES',
                                                 'M_MAX','ING_BRUTO'])
df_scaled

In [ ]:
# Visualizando escalado
sns.kdeplot(data = df_scaled)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,15))
ax=fig.gca()
df_scaled.plot(ax = ax, kind = 'density', subplots = True, layout = (5,4), sharex = False)
plt.show()

In [ ]:
df_target_array = pd.DataFrame(data=Y_clas,columns = ['Default'])
df_target_array

In [ ]:
# Concatenando datos escalados con variable target
df_consolidado = pd.concat([df_scaled, df_target_array], axis = 1)
cols=df_consolidado.columns.tolist()
cols

In [ ]:
df_consolidado.sample(10)

In [ ]:
fig = plt.figure(figsize = (15,15))
ax =fig.gca()
df_consolidado.plot(ax = ax, kind = 'box', subplots = True, layout =(6,6), sharex = False)
plt.show()

**Análisis multivariado**

In [ ]:
#Correlación de Pearson basado en métodos estadísticos y análisis de correlación
df_corr=df_consolidado.copy()
correlation = df_corr.corr(method ='pearson')
correlation #arroja matriz de correlaciones

In [ ]:
df_corr.info()

In [ ]:
matcorr=df_corr.corr()
plt.figure(figsize=(15,15))
ax= sns.heatmap(matcorr, vmax=1, square = True,annot = True, cmap = "Dark2")
plt.title("Matriz de Correlaciones", fontsize = 18)
plt.show

In [ ]:
df_corr['Default'] = df_corr['Default'].astype(int)

In [ ]:
matcorr1=df_corr.corr()
plt.figure(figsize=(15,15))
ax= sns.heatmap(matcorr1, vmax=1, square = True,annot = True, cmap = "Dark2")
plt.title("Matriz de Correlaciones", fontsize = 18)
plt.show

In [ ]:
relevant_features=matcorr1[matcorr1>0.2]
relevant_features

In [ ]:
cor_target =abs(matcorr1['Default'])
cor_target

**Selección de mejores features**

In [ ]:
cor_target = abs(matcorr1['Default'])
relevant_features = cor_target[cor_target>0.05]
relevant_features

In [ ]:
#Cargue de librerías Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

In [ ]:
df_features = df_consolidado.copy()
array = df_features.values
X_class = array[ : ,0:9]
Y_class = array[ : , 10]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split (X_class, Y_class, test_size=0.3, random_state = 7)

depth = 5
tree = DecisionTreeRegressor(criterion = 'friedman_mse', max_depth = depth)
tree.fit(X_train, Y_train)

#Extayendo índices
subset =np.unique(tree.tree_.feature[tree.tree_.feature>=0])

print(f'Total Variables: {X_class.shape[1]}')
print(f'Variables utilizadas: {subset}')
print(f'Entrenamiento: {tree.score(X_train, Y_train).round(2)}')
print(f'Test: {tree.score(X_test, Y_test).round(2)}')

In [ ]:
df_features.sample(4)

In [ ]:
ranking = np.argsort(tree.feature_importances_)
print(df_features.columns[ranking])

In [ ]:
from importlib import reload
plt=reload(plt)

importances=tree.feature_importances_

plt.figure(figsize = (8,6))
plt.title('Ranking Variables mas importantes')
plt.barh(range(X_class.shape[1]), importances[ranking])
plt.yticks(range(X_class.shape[1]),df_features.columns[ranking], fontsize=9)
plt.show()

**Selección y validación del modelo**

In [ ]:
df_features.info()

In [ ]:
df_features['Default'] = df_features['Default'].astype(int)

In [ ]:
array = df_features.values
X_class1 = array[ : , 0:9]
Y_class1 = array[ : , 10]
X_train, X_test, Y_train, Y_test = train_test_split (X_class1, Y_class1, test_size=0.2, random_state = 7)

In [ ]:
test_size = 0.3
seed = 7

#Creando modelo de regresión logística

model = LogisticRegression(solver='lbfgs', max_iter=1000)
model.fit(X_train, Y_train)
results = model.score(X_test,Y_test)
print(f'Accuracy_Logistic_Regressor: {results*100.0:,.2f}%')

**Probando más modelos**

In [ ]:
#Importando librerías con diversos modelos

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

In [ ]:
models = []
models.append(('LoR',LogisticRegression(solver='lbfgs', max_iter=1000)))
models.append(('K-NN',KNeighborsClassifier()))
models.append(('SVC',SVC(gamma ='auto')))
models.append(('NB',GaussianNB()))

In [ ]:
results1=[]
names=[]
scoring = "accuracy"

In [ ]:
for name, model in models:
  kfold = KFold(n_splits = 10)
  cv_results = cross_val_score(model, X_class1, Y_class1, cv=kfold, scoring = scoring)
  results1.append(cv_results)
  names.append(name)

In [ ]:
fig = plt.figure()
fig.suptitle("Comparación de Algorítmos")
ax=fig.add_subplot(111)
plt.boxplot(results1)
ax.set_xticklabels(names)
plt.show()

El Gaussian Naive Bayes (GNB) es una técnica de clasificación utilizada en el aprendizaje automático basada en un enfoque probabilístico y una distribución gaussiana. Gaussian Naive Bayes asume que cada parámetro, también llamado características o predictores, tiene una capacidad independiente de predecir la variable de salida.

Es una técnica de clasificación de aprendizaje automático basada en un enfoque probabilístico que supone que cada clase sigue una distribución normal. Asume que cada parámetro tiene una capacidad independiente de predecir la variable de salida. Es capaz de predecir la probabilidad de que una variable dependiente sea clasificada en cada grupo.

https://builtin.com/artificial-intelligence/gaussian-naive-bayes